# APLICACIONES EN CIENCIAS DE COMPUTACION

## Laboratorio 3:  Busqueda en Laberintos

La tarea de este laboratorio consiste en comparar los dos métodos principales de busqueda ciega para buscar caminos en un laberinto. La clase MazeSearchProblem necesita ser completada (metodos actions y result). Después de implementar ello pruebe las implementaciones con el laberinto maze3.txt. Al final de este notebook estan las preguntas que seran evaluadas en este laboratorio. 

# ***Usted deberá completar el código en las secciones indicadas con "COMPLETAR"***


### Clase <b>Maze</b>
La clase Maze es representada por una matriz (grid) de espacios, los cuales pueden estar libres o bloqueados. Cada celda del grid puede tener uno de los siguientes carateres:

    '#': Indica una celda con obstaculo  (impasable) 
    '~': Indica una celda con agua (pasable, con costo 7)
    ' ': Celda vacia (passable con costo 2)
    'S': Indicate la celda de inicio
    'E': Indicates la celda de salida 

In [1]:
# ESTA CELDA NO NECESITA SER MODIFICADA

class Maze:

    def __init__(self, grid):
        """ Construye el maze a partir del grid pasado
            grid: debe ser una matriz (lista de listas), ejemplo [['#','S',' '],['#',' ','E']]  """
        self.grid = grid
        self.numRows = len(grid)
        self.numCols = len(grid[0])
        for i in range(self.numRows):
            for j in range(self.numCols):
                if len(grid[i]) != self.numCols:
                    raise "Grid no es Rectangular"
                if grid[i][j] == 'S':
                    self.startCell = (i,j)
                if grid[i][j] == 'E':
                    self.exitCell= (i,j)
        if self.exitCell == None:
            raise "No hay celda de Inicio"
        if self.startCell == None:
            raise "No hay celda de salida"
   
    def isPassable(self, row, col):
        """ Retorna true si la celda (row,col) es pasable  (' ' o '~') """
        return self.isWater(row, col) or self.isClear(row, col)   

    def isWater(self, row, col):
        """ Retorna true si la celda (row,col) tiene agua  ('~') """
        return self.grid[row][col] == '~'
    
    def isClear(self, row, col):
        """ Retorna true si la celda (row,col) esta vacia  (' ') """
        return self.grid[row][col] == ' '
    
    def isBlocked(self, row,col):
        """ Retorna true si la celda (row,col) tiene obstaculo ('#') """
        return self.grid[row][col] == '#'   
        
    def getNumRows(self):
        """ Retorna el numero de filas en el maze """
        return self.numRows
  
    def getNumCols(self):
        """ Retorna el numero de columnas en el maze """
        return self.numCols  
   
    def getStartCell(self):
        """ Retorna la posicion (row,col) de la celda de inicio """
        return self.startCell
  
    def getExitCell(self):
        """ Retorna la posicion (row,col) de la celda de salida """
        return self.exitCell

    def __getAsciiString(self):
        """ Retorna el string de vizualizacion del maze """
        lines = []
        headerLine = ' ' + ('-' * (self.numCols)) + ' '
        lines.append(headerLine)
        for row in self.grid:
            rowLine = '|' + ''.join(row) + '|'
            lines.append(rowLine)
        lines.append(headerLine)
        return '\n'.join(lines)

    def __str__(self):
        return self.__getAsciiString()

### Funcion para cargar una laberinto de archivo de disco

In [2]:
# ESTA CELDA NO NECESITA SER MODIFICADA
def readMazeFromFile(file):
    """ Lee un archivo que contiene un laberinto y retorna una instancia de Maze con dicho laberinto"""
    fin = open(file)
    lines = fin.read().splitlines()
    grid = []
    for line in lines: 
        grid.append(list(line))
    return Maze(grid)

### Clase <b>SearchProblem</b>

Esta es una clase abstracta para definir problemas de busqueda. Se debe hacer subclases que implementen los metodos de las acciones, resultados, test de objetivo y el costo de camino. Entonces se puede instanciar las subclases y resolverlos con varias funciones de busqueda.

In [3]:
# ESTA CELDA NO NECESITA SER MODIFICADA
class SearchProblem(object):
    def __init__(self, initial, goal=None):
        """Este constructor especifica el estado inicial y posiblemente el estado(s) objetivo(s),
        La subclase puede añadir mas argumentos."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Retorna las acciones que pueden ser ejecutadas en el estado dado.
        El resultado es tipicamente una lista."""
        raise NotImplementedError

    def result(self, state, action):
        """Retorna el estado que resulta de ejecutar la accion dada en el estado state.
        La accion debe ser alguna de self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Retorna True si el estado pasado satisface el objetivo."""
        raise NotImplementedError

    def path_cost(self, c, state1, action, state2):
        """Retorna el costo del camino de state2 viniendo de state1 con 
        la accion action, asumiendo un costo c para llegar hasta state1. 
        El metodo por defecto cuesta 1 para cada paso en el camino."""
        return c + 1

###  <b> Clase MazeSearchProblem </b>  
Esta es una subclase de SearchProblem que implementa concretamente el problema de busqueda en laberinto. El constructor recibe el laberinto en un objeto maze. Cada estado es codificado como una tupla (row,col) representando la posicion de una celda del grid. El estado inicial es la celda de inicio y el único estado objetivo es la celda de salida.
###  <b>  </b>  
 **Se necesita completar Actions (acciones legales para un estado dado - Considerar las acciones  'N', 'S', 'W', 'E') y result (qué hacen las acciones).**

In [5]:
# ESTA CELDA REQUIERE COMPLETAR CÓDIGO DONDE SE INDIQUE "COMPLETAR"
class MazeSearchProblem(SearchProblem):
    def __init__(self, maze):
        """El constructor recibe el maze"""
        self.maze = maze
        self.initial = maze.getStartCell()
        self.goal = maze.getExitCell()
        self.numNodesExpanded = 0        
        self.expandedNodeSet = {}   
        
    def __isValidState(self,state):
        """ Retorna true si el estado dado corresponde a una celda no bloqueada valida """
        row,col = state
        if row < 0 or row >= self.maze.getNumRows():
            return False
        if col < 0 or col >= self.maze.getNumCols():
            return False
        return not self.maze.isBlocked(row,col)        

    def actions(self, state):
        """Retorna las acciones legales desde la celda actual """
        row,col = state
        acciones = []
        if self.__isValidState((row-1, col)):
            acciones.append('N')

        ## COMPLETAR
        # verificar que no nos pasemos de los bordes
        if self.__isValidState((row+1, col)):
            acciones.append('S')
        if self.__isValidState((row, col + 1)):
            acciones.append('E')
        if self.__isValidState((row, col - 1)):
            acciones.append('W')
        ## FIN DE SU CÓDIGO AÑADIDO
            
        return acciones
    
    def result(self, state, action):
        """Retorna el estado que resulta de ejecutar la accion dada desde la celda actual.
        La accion debe ser alguna de self.actions(state)"""  
        row,col = state
        newState = ()
        if action == 'N':
            newState = (row-1, col)

        ## COMPLETAR
        if action == 'S':
            newState = (row+1, col)
        if action == 'E':
            newState = (row, col+1)
        if action == 'W':
            newState = (row, col-1)
        ## FIN DE SU CÓDIGO AÑADIDO

        return newState
        
    def goal_test(self, state):
        """Retorna True si state es self.goal"""
        return (self.goal == state) 

    def path_cost(self, c, state1, action, state2):
        """Retorna el costo del camino de state2 viniendo de state1 con la accion action 
        El costo del camino para llegar a state1 es c. El costo de la accion sale de self.maze """
        row, col = state2
        if self.maze.isClear(row, col):
            actionCost =   2
        elif self.maze.isWater(row, col):
            actionCost =   7
        elif state2 == self.maze.getStartCell() or state2 == self.maze.getExitCell():
            actionCost = 1
        else:
            raise "El costo de una celda bloqueada no esta definido" 
        return c + actionCost

### Clase <b>Node</b>

Estructura de datos para almacenar la informacion de un nodo en un <b>arbol de busqueda</b>. Contiene información del nodo padre y el estado que representa el nodo. Tambien incluye la accion que nos llevo al presente nodo y el costo total del camino desde el nodo raiz hasta este nodo.

In [6]:
# ESTA CELDA NO NECESITA SER MODIFICADA
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        "Crea un nodo de arbol de busqueda, derivado del nodo parent y accion action"
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def expand(self, problem):
        "Devuelve los nodos alcanzables en un paso a partir de este nodo."
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        next = problem.result(self.state, action)
        return Node(next, self, action,
                    problem.path_cost(self.path_cost, self.state, action, next))

    def solution(self):
        "Retorna la secuencia de acciones para ir de la raiz a este nodo."
        return [node.action for node in self.path()[1:]]

    def path(self):
        "Retorna una lista de nodos formando un camino de la raiz a este nodo."
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
    
    def __lt__(self, node):
        return self.state < node.state
    
    def __eq__(self, other): 
        "Este metodo se ejecuta cuando se compara nodos. Devuelve True cuando los estados son iguales"
        return isinstance(other, Node) and self.state == other.state
    
    def __repr__(self):
        return "<Node {}>".format(self.state)
    
    def __hash__(self):
        return hash(self.state)

### <b> Frontera tipo cola FIFO (first-in first out)</b> 

In [7]:
# ESTA CELDA NO NECESITA SER MODIFICADA
from collections import deque

class FIFOQueue(deque):
    """Una cola First-In-First-Out"""
    def pop(self):
        return self.popleft()

### <b>Algoritmo general de búsqueda con memoria de nodos expandidos (Graph Search)</b>

Algoritmo de general de busqueda ciega con memoria de estados visitados. El argumento frontier debe ser una cola vacia. Si la frontera es tipo FIFO hace busqueda en amplitud (BFS), si la frontera es una pila hará busqueda en profundidad (DFS)

In [8]:
# ESTA CELDA NO NECESITA SER MODIFICADA
def graph_search(problem, frontier):
    frontier.append(Node(problem.initial))
    explored = set()     # memoria de estados visitados
    visited_nodes = []   # almacena nodos visitados durante la busqueda
    while frontier:
        node = frontier.pop()
        visited_nodes.append(node)
        if problem.goal_test(node.state):
            return node, visited_nodes
        explored.add(node.state)
        
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and
                        child not in frontier)
    return None

### <b> Funcion para mostrar los resultados </b> 

In [9]:
# ESTA CELDA NO NECESITA SER MODIFICADA
def displayResults(maze, visitedNodes, solutionNodes):
    """ Muestra los resultados de busqueda en el maze.   """
    grid_copy = []
    for row in maze.grid:
        grid_copy.append([x for x in row]) 
    for node in visitedNodes:
        row,col = node.state
        ch = maze.grid[row][col]
        if ch != 'S' and ch != 'E': grid_copy[row][col] = 'o' 
    for node in solutionNodes:  
        row,col = node.state
        ch = maze.grid[row][col]
        if ch != 'S' and ch != 'E': grid_copy[row][col] = 'x'    
    maze_copy = Maze(grid_copy)
    print (maze_copy)
    print ("x - celdas en la solucion")
    print ("o - celdas visitadas durante la busqueda")
    print ("-------------------------------")

## <b> Experimentación con los algoritmos de Busqueda</b> 


In [10]:
# ESTA CELDA NO NECESITA SER MODIFICADA
""" Carga un laberinto de archivo en disco e instancia el problema de busqueda.   """
maze = readMazeFromFile('maze3.txt') 
p = MazeSearchProblem(maze)
print(maze)

 ------------------- 
|~~~~S~~~~~~~~~~~~~~|
|~~~~~~~~~~~~~~~~~~~|
|~~~~~          ~~~~|
|~~~~            ~~~|
|########   ~~~~~~~~|
|             ######|
|      #      #~~~~~|
|###### # #####~~~~~|
|     # #      ~~~~~|
| # # ###### ####~~~|
| #          #~~~~~~|
|     ### ####~~~~~~|
|                  E|
 ------------------- 


### busqueda en profundidad (DFS)

In [12]:
nsol, visited_nodes = graph_search(p, [])
print('Solucion DFS: {}. Nodos visitados={}. Costo Solucion = {}'.format(nsol.solution(), len(visited_nodes),nsol.path_cost))
displayResults(maze, visited_nodes, nsol.path())

Solucion DFS: ['W', 'W', 'W', 'W', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'S', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'W', 'W', 'W', 'W', 'W', 'S', 'S', 'E', 'E', 'E', 'E', 'E']. Nodos visitados=122. Costo Solucion = 307
 ------------------- 
|xxxxS~~~~~~~~~~~~~~|
|x~~~~~~~~~~~~~~~~~~|
|xxxxxxxxxxxxxxxxxxx|
|~~~~            ~~x|
|########xxxxxxxxxxx|
|oooooooox    ######|
|oooooo#oxoooo#~~~~~|
|###### #x#####~~~~~|
|ooooo# #xxxxxxxxxxx|
|o#o#o###### ####~~x|
|o#oooooooooo#xxxxxx|
|ooooo###o####x~~~~~|
|oooooooooooooxxxxxE|
 ------------------- 
x - celdas en la solucion
o - celdas visitadas durante la busqueda
-------------------------------


### busqueda en amplitud (BFS)

In [16]:
nsol, visited_nodes = graph_search(p, FIFOQueue())
print('Solucion DFS: {}. Nodos visitados={}. Costo Solucion = {}'.format(nsol.solution(), len(visited_nodes),nsol.path_cost))
displayResults(maze, visited_nodes, nsol.path())

Solucion DFS: ['S', 'S', 'S', 'E', 'E', 'E', 'E', 'S', 'S', 'S', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'S', 'S', 'E', 'E']. Nodos visitados=177. Costo Solucion = 91
 ------------------- 
|ooooSoooooooooooooo|
|ooooxoooooooooooooo|
|ooooxoooooooooooooo|
|ooooxxxxxoooooooooo|
|########xoooooooooo|
|ooooooooxoooo######|
|oooooo#oxoooo#ooooo|
|###### #x#####ooooo|
|    o# #xxxxxxxxxoo|
| # #o######o####xoo|
| #oooooooooo#oooxoo|
|   oo###o####~ooxoo|
|    ooooooooo  oxxE|
 ------------------- 
x - celdas en la solucion
o - celdas visitadas durante la busqueda
-------------------------------


## Preguntas
<b>1) Completar el código (6 puntos)  </b>
 
<b>2) En cuanto a los resultados obtenidos. ¿Los algoritmos BFS o DFS encuentran soluciones optimas? ¿Por Qué? Explique detalladamente (4 puntos)</b>

<b>3) Compare los resultados obtenidos por BFS y DFS en cuanto a nodos visitados y costo de solución. Explique la razón de las diferencias o similitudes. (4 puntos) </b>

<b>4) ¿Qué diferencia consigue notar entre las regiones exploradas por los dos métodos de búsqueda? Explique con referencia a las celdas visitadas durante la búsqueda y cómo el método de búsqueda llega a visitar dichas celdas. (3 puntos)</b>

<b>5) Para el problema del laberinto desarrollado en este laboratorio. ¿Qué ventaja habría si en vez de hacer busqueda BFS con la implementacion graph_search  se hubiera implementado la funcion BREADTH-FIRST-SEARCH  (diapositiva 8  de  BusquedaSinInformacion.pdf)? (3 puntos)</b>



 

**Pregunta 2** <br>
Ambos algoritmos obtienen soluciones optimas ya que logran llegar del punto S al punto E y evitan las posiciones con obstaculos impasables.

**Pregunta 3** <br>
Ambos algoritmos encuentran soluciones, pero el algoritmo que encuentra la solucion mas optima es el algoritmo BFS ya que tiene un costo de 91 a comparacion del costo del algoritmo DFS el cual tiene un costo de 307. El algoritmo DFS se mueve a travez de menos nodos mientras que el algoritmo DFS se mueve a travez de mas nodos.

**Pregunta 4** <br>
El algoritmo DFS trata de ir de un extremo del mapa hasta el opuesto en cada movimiento hasta que encuentra un obstaculo. Esto se debe a que el algoritmo DFS utiliza una pila como frontera, por lo que siempre va a favorecer los nodos mas nuevos a los nodos mas antiguos por lo que da una solucion no muy buena. El algoritmo BFS es mas directo puesto que el algoritmo no realiza tantos movimientos ya que este usa una FIFOQueue como frontera, esto beneficia ya que siempre va a favorecer a los nodos mas antiguos a comparacion de los nodos mas recientes, esto permite que encuentre la solucion mas rapido.

**Pregunta 5**<br>
Ninguna ya que la funcion graph_search desarrollada en este laboratorio nos da flexibilidad puesto que uno puede utilizar DFS o BFS dependiendo del tipo de frontera pasada a la funcion, permitiendonos decidir el mejor algoritmo para el problema. En cambio, la funcion de las diapositivas solo utiliza un FIFOQueue como frontera por lo que no tenemos flexibilidad.